In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import numpy as np
import torch

import cv2
from PIL import Image

# download an image
image = load_image(
    "https://web.engr.oregonstate.edu/~huntchri/connor.png"
)
image = np.array(image)

# get canny image
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

# load control net and stable diffusion v1-5
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)

# speed up diffusion process with faster scheduler and memory optimization
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.enable_xformers_memory_efficient_attention()

pipe.enable_model_cpu_offload()

# generate image
generator = torch.manual_seed(0)
image = pipe(
    "pucker baby frankenstein", num_inference_steps=20, generator=generator, image=canny_image, guidance_scale=50, num_images_per_prompt=2
).images

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

# Display image1
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
plt.axis('off')
plt.imshow(image[0])

# Display image2
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
plt.axis('off')
plt.imshow(image[1])

# Display image3
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
plt.axis('off')
plt.imshow(canny_image)

# Show all sub-plots
plt.show()